# Create Simple Dataset

In [611]:
import cv2
import os

In [612]:
card_simple_rect = (30, 40, 130, 210)
card_simple_factor = 2
original_database_foldername = "resized"
name_database_foldername = "names"
suit_database_foldername = "suits"
simple_database_foldername = "simple"
img_type = "png"

In [613]:
def cnt1_is_inside_cnt2(cnt1, cnt2):
    cnt1_x1, cnt1_y1, cnt1_w, cnt1_h = cv2.boundingRect(cnt1)
    cnt1_x2 = cnt1_x1 + cnt1_w
    cnt1_y2 = cnt1_y1 + cnt1_h
    cnt2_x1, cnt2_y1, cnt2_w, cnt2_h = cv2.boundingRect(cnt2)
    cnt2_x2 = cnt2_x1 + cnt2_w
    cnt2_y2 = cnt2_y1 + cnt2_h

    return (cnt1_x1 >= cnt2_x1) and (cnt1_y1 >= cnt2_y1) and (cnt1_x2 <= cnt2_x2) and (cnt1_y2 <= cnt2_y2)

In [614]:
def get_card_simple_area():
    x1, y1, x2, y2 = card_simple_rect
    return (x2 - x1) * (y2 - y1) * card_simple_factor ** 2

In [615]:
def extract_card_simple(card_img):
    x1, y1, x2, y2 = card_simple_rect
    card_simple = card_img[y1:y2, x1:x2]
    card_simple = cv2.resize(card_simple, (0,0), fx=card_simple_factor, fy=card_simple_factor) 
    return card_simple


'\ndef extract_card_name_suit_images(card_img):\n    x1, y1, x2, y2 = card_simple_rect\n    h = y2 - y1\n    card_img_simple = card_img[y1:y2, x1:x2]\n    name_img = card_img_simple[:h//2,:]\n    suit_img = card_img_simple[h//2+1:,:]\n    return name_img, suit_img\n'

In [616]:
def extract_card_name_suit(filename):
    #suit_legend = {"C": "Clubs", "D": "Diamonds", "H": "Hearts", "S": "Spades"}
    name = filename.split(".")[0]
    name_split = name.split("_")
    return name_split[0], name_split[1]

In [617]:
def create_binary(frame):
    # convert to gray
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # apply blur
    frame = cv2.GaussianBlur(frame, (3,3), 0)

    # convert to binary
    frame = cv2.threshold(frame, 210, 255, cv2.THRESH_BINARY)[1]

    return frame

In [618]:
def apply_morphological_dilatation(binary):
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (19, 3))
    dilation = cv2.erode(binary, rect_kernel, iterations = 1)
    return dilation

In [619]:
def find_contourns(binary):
    cnts = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    return cnts

In [620]:
def filter_contourns(cnts):
    candidates = []
    candidates_to_dups = []
    simple_x1, simple_y1, simple_x2, simple_y2 = card_simple_rect
    simple_w = (simple_x2 - simple_x1) * card_simple_factor
    simple_h = (simple_y2 - simple_y1) * card_simple_factor

    for cnt in cnts:
        cnt_x, cnt_y, cnt_w, cnt_h = cv2.boundingRect(cnt)
        area_cnt = cnt_w * cnt_h

        # Area ration verification
        if area_cnt / get_card_simple_area() > 0.7:
            continue

        # Position verification
        if cnt_x / simple_w > 0.5:
            continue

        candidates_to_dups.append(cnt)

    # Duplicates
    for i in range(len(candidates_to_dups)):
        is_inside = False
        for j in range(len(candidates_to_dups)):
            if i != j:
                cnt1 = candidates_to_dups[i]
                cnt2 = candidates_to_dups[j]
                if cnt1_is_inside_cnt2(cnt1, cnt2):
                    is_inside = True
                    break
        if not is_inside:
            candidates.append(cnt1)

    return candidates

In [621]:
def extract_card_name_suit_images_from_contourns(simple_img, cnts):
    name_cnt = cnts[1]
    suit_cnt = cnts[0]

    name_cnt_x, name_cnt_y, name_cnt_w, name_cnt_h = cv2.boundingRect(name_cnt)
    suit_cnt_x, suit_cnt_y, suit_cnt_w, suit_cnt_h = cv2.boundingRect(suit_cnt)

    name_img = simple_img[name_cnt_y:name_cnt_y+name_cnt_h, name_cnt_x:name_cnt_x+name_cnt_w]
    suit_img = simple_img[suit_cnt_y:suit_cnt_y+suit_cnt_h, suit_cnt_x:suit_cnt_x+suit_cnt_w]

    return name_img, suit_img

In [622]:
def extract_card_name_suit_images(img):
    simple_img = extract_card_simple(img)
    binary_simple_img = create_binary(simple_img)
    simple_img_cnts = find_contourns(binary_simple_img)
    simple_img_cnts = filter_contourns(simple_img_cnts)
    name_img, suit_img = extract_card_name_suit_images_from_contourns(binary_simple_img, simple_img_cnts)
    return name_img, suit_img

In [623]:
# Create folder to store images related to names of the cards
if not os.path.exists(name_database_foldername):
   os.makedirs(name_database_foldername)

# Create folder to store images related to suits of the cards
if not os.path.exists(suit_database_foldername):
   os.makedirs(suit_database_foldername)

# Create folder to store simple version of cards
if not os.path.exists(simple_database_foldername):
   os.makedirs(simple_database_foldername)

filenames = os.listdir(original_database_foldername)
i = 0
for filename in filenames:
    name, suit = extract_card_name_suit(filename)
    img = cv2.imread(f"./{original_database_foldername}/{filename}")

    #simple_img = extract_card_name_suit_images(img)
    simple_img = extract_card_simple(img)
    binary_simple_img = create_binary(simple_img)
    simple_img_cnts = find_contourns(binary_simple_img)
    simple_img_cnts = filter_contourns(simple_img_cnts)
    name_img, suit_img = extract_card_name_suit_images_from_contourns(binary_simple_img, simple_img_cnts)
    #name_img, suit_img = extract_card_name_suit_images(img)
    if (not os.path.exists(f"{name_database_foldername}/{name}.{img_type}")):
        cv2.imwrite(f"{name_database_foldername}/{name}.{img_type}", name_img)
    if (not os.path.exists(f"{suit_database_foldername}/{suit}.{img_type}") and i > 1):
        cv2.imwrite(f"{suit_database_foldername}/{suit}.{img_type}", suit_img)
    i += 1

cv2.waitKey(0)
cv2.destroyAllWindows()